In [228]:
import os
import pandas as pd
import numpy as np
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint

In [229]:
#os.chdir('..')
os.chdir('C:/Users/BC966HL/OneDrive - EY/4. Mestrado/Dev/')
os.getcwd()

'C:\\Users\\BC966HL\\OneDrive - EY\\4. Mestrado\\Dev'

In [230]:
df_recipe = pd.read_csv('Datasets/model/input_recipe.csv', sep = ';')
df_available_lots = pd.read_csv('Datasets/model/input_lots_available.csv', sep = ';')

In [231]:
from pandas_profiling import ProfileReport

#ProfileReport(df_available_lots)

In [232]:
# Recipe constraint considering delta of 1 between parameters reference and found
const_aroma = NonlinearConstraint(lambda x: abs(float(df_recipe['AROMA']) - (sum(np.array(df_available_lots['AROMA'])*x))/sum(x)), 0, 1)
const_flavor = NonlinearConstraint(lambda x: abs(float(df_recipe['FLAVOR']) - (sum(np.array(df_available_lots['FLAVOR'])*x))/sum(x)), 0, 1)
const_aftertaste = NonlinearConstraint(lambda x: abs(float(df_recipe['AFTERTASTE']) - (sum(np.array(df_available_lots['AFTERTASTE'])*x))/sum(x)), 0, 1)
const_acidity = NonlinearConstraint(lambda x: abs(float(df_recipe['ACIDITY']) - (sum(np.array(df_available_lots['ACIDITY'])*x))/sum(x)), 0, 1)
const_body = NonlinearConstraint(lambda x: abs(float(df_recipe['BODY']) - (sum(np.array(df_available_lots['BODY'])*x))/sum(x)), 0, 1)
const_balance = NonlinearConstraint(lambda x: abs(float(df_recipe['BALANCE']) - (sum(np.array(df_available_lots['BALANCE'])*x))/sum(x)), 0, 1)
#const_uniformity = NonlinearConstraint(abs(df_recipe['UNIFORMITY'] - (sum(np.array(df_available_lots['UNIFORMITY'])*m))/sum(m)), 0, 1)
#const_clean_cup = NonlinearConstraint(abs(df_recipe['CLEAN_CUP'] - (sum(np.array(df_available_lots['CLEAN_CUP'])*m))/sum(m)), 0, 1)
#const_sweetness = NonlinearConstraint(abs(df_recipe['SWEETNESS'] - (sum(np.array(df_available_lots['SWEETNESS'])*m))/sum(m)), 0, 1)
const_moisture = NonlinearConstraint(lambda x: abs(float(df_recipe['MOISTURE']) - (sum(np.array(df_available_lots['MOISTURE'])*x))/sum(x)), 0, .01)
const_volume_need = NonlinearConstraint(lambda x: sum(x), .95*float(df_recipe['VOLUME_NEED']), 1.05*float(df_recipe['VOLUME_NEED']))

In [233]:
# Objective functions
# Main OF
fo_cost = sum(m*np.array(df_available_lots['COST_BRL_KG']))

# Parameters delta minimization
fo_aroma = df_recipe['AROMA'] - (sum(np.array(df_available_lots['AROMA'])*m))/sum(m)
fo_flavor = df_recipe['FLAVOR'] - (sum(np.array(df_available_lots['FLAVOR'])*m))/sum(m)
fo_aftertaste = df_recipe['AFTERTASTE'] - (sum(np.array(df_available_lots['AFTERTASTE'])*m))/sum(m)

NameError: name 'm' is not defined

In [234]:
# Creating bound matrix with xn with lb = 0 and ub = lot n availability
bnd = []
for limit in df_available_lots['LOT_AVAILABILITY_KG']:
    bnd.append((0, int(limit)))

In [235]:
def cost_obj_f(x, cost_per_lot):
    return np.sqrt(sum(np.array(cost_per_lot)*x))

In [236]:
#x = np.ones(len(df_available_lots['COST_BRL_KG']))
slsqp.status = -1
while(slsqp.status != 0):
    slsqp = minimize(cost_obj_f,
                x0 = np.random.randint(0, 100, len(df_available_lots['COST_BRL_KG'])),
                args = (df_available_lots['COST_BRL_KG'],),
                method = 'SLSQP',
                constraints = [const_volume_need, const_aroma, const_flavor, const_aftertaste, const_acidity, const_body, const_balance, const_moisture],
                bounds = bnd
                #constraints = [const_aroma, const_flavor, const_aftertaste, const_acidity, const_body, const_balance, const_uniformity, const_clean_cup, const_sweetness,                                  const_moisture, const_volume_per_lot, const_volume_need]
                )

slsqp.message, sum(slsqp.x), float(df_recipe['VOLUME_NEED']), slsqp.fun**2

('Optimization terminated successfully.',
 9138.04999999941,
 9619.0,
 129695.41181123425)

In [202]:
#x = np.ones(len(df_available_lots['COST_BRL_KG']))
cobyla = minimize(cost_obj_f,
                x0 = np.random.randint(0, 100, len(df_available_lots['COST_BRL_KG'])),
                args = (df_available_lots['COST_BRL_KG'],),
                method = 'COBYLA',
                constraints = [const_volume_need, const_aroma, const_flavor, const_aftertaste, const_acidity, const_body, const_balance, const_moisture],
                bounds = bnd
                #constraints = [const_aroma, const_flavor, const_aftertaste, const_acidity, const_body, const_balance, const_uniformity, const_clean_cup, const_sweetness,                                  const_moisture, const_volume_per_lot, const_volume_need]
                )

cobyla.message, sum(cobyla.x), float(df_recipe['VOLUME_NEED']), cobyla.fun**2

('Did not converge to a solution satisfying the constraints. See `maxcv` for magnitude of violation.',
 3608.4926059065265,
 66479.0,
 105100.02000304202)

In [11]:
#Recipe constraints, considering 5% lower and upper bounds on the df_recipe parameters
#const_aroma = LinearConstraint([1], .95*float(df_recipe['AROMA']), 1.05*float(df_recipe['AROMA']))
#const_flavor = LinearConstraint([1], .95*float(df_recipe['FLAVOR']), 1.05*float(df_recipe['FLAVOR']))
#const_aftertaste = LinearConstraint([1], .95*float(df_recipe['AFTERTASTE']), 1.05*float(df_recipe['AFTERTASTE']))
#const_acidity = LinearConstraint([1], .95*float(df_recipe['ACIDITY']), 1.05*float(df_recipe['ACIDITY']))
#const_body = LinearConstraint([1], .95*float(df_recipe['BODY']), 1.05*float(df_recipe['BODY']))
#const_balance = LinearConstraint([1], .95*float(df_recipe['BALANCE']), 1.05*float(df_recipe['BALANCE']))
#const_uniformity = LinearConstraint([1], .95*float(df_recipe['UNIFORMITY']), 1.05*float(df_recipe['UNIFORMITY']))
#const_clean_cup = LinearConstraint([1], .95*float(df_recipe['CLEAN_CUP']), 1.05*float(df_recipe['CLEAN_CUP']))
#const_sweetness = LinearConstraint([1], .95*float(df_recipe['SWEETNESS']), 1.05*float(df_recipe['SWEETNESS']))
#const_moisture = LinearConstraint([1], .95*float(df_recipe['MOISTURE']), 1.05*float(df_recipe['MOISTURE']))
#const_volume_need = LinearConstraint([1], .95*float(df_recipe['VOLUME_NEED']), 1.05*float(df_recipe['VOLUME_NEED']))